In [28]:
from model_python import *
from data_preprocess import * 
from textblob import TextBlob
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
tweetdata=pd.read_csv("data/trumptweets.csv")
tweetdata['date'] = pd.to_datetime(tweetdata['date'])
tweetdata['time'] = tweetdata['date'].dt.time
tweetdata['hour'] = tweetdata['date'].dt.strftime('%H')
tweetdata['min'] = tweetdata['date'].dt.strftime('%M')
tweetdata['hour']=pd.to_numeric(tweetdata['hour'])
tweetdata['min']=pd.to_numeric(tweetdata['min'])
tweetdata['hour']=tweetdata['hour']+(tweetdata['min'])/60
tweetdata['date'] = pd.to_datetime(tweetdata['date']).dt.strftime('%m/%d/%Y')
tre_rate_data=pd.read_table("data/YieldCurve.txt",sep="\t")
tre_rate_data['Date'] = pd.to_datetime(tre_rate_data['Date']).dt.strftime('%m/%d/%Y')
Tweet_rate_new = pd.merge(left=tweetdata , right=tre_rate_data, left_on='date', right_on='Date')
Tweet_rate_new=Tweet_rate_new.drop(['id','link','retweets','favorites','mentions','hashtags','geo','Date','min','time'],axis=1)
import numpy as np
df1=np.array(Tweet_rate_new['1 Mo'])
df1_log=[]
for x in range(len(df1)-1):
    if Tweet_rate_new['hour'][x]>15:
        if df1[x+1]>df1[x]:
            df1_log.append('up')
        elif df1[x+1]<df1[x]:
            df1_log.append('down')
        else:
            df1_log.append('neutral')
    else:
        if df1[x]>df1[x-1]:
            df1_log.append('up')
        elif df1[x]<df1[x-1]:
            df1_log.append('down')
        else:
            df1_log.append('neutral')
df11_log=df1[1:]-df1[:-1]
Tweet_rate_new=Tweet_rate_new[1:]
Tweet_rate_new['Label']=df1_log
Tweet_rate_new['1 Mo pric diff']=df11_log
Tweet_rate_new = Tweet_rate_new[['date','hour','content','1 Mo','Label','1 Mo pric diff']]

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead.
  # This is added back by InteractiveShellApp.init_path()


In [3]:
Tweet_rate=Tweet_rate_new
# Load the regular expression library
import re
# Remove punctuation
Tweet_rate['content_processed'] = Tweet_rate['content'].map(lambda x: re.sub('[^\\w\\s]', ' ', x))
# Convert the titles to lowercase
Tweet_rate['content_processed'] = Tweet_rate['content_processed'].map(lambda x: x.lower())

'''
clean the text in a tweet by removing links and special characters using regex.
'''
Tweet_rate['content_processed'] = Tweet_rate['content_processed'].map(lambda x: ' '.join(re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", x).split()))
  
# Remove stopwords
with open("./data/stopwords.txt") as f:
    stopwords = f.read()
Tweet_rate['content_processed'] = [' '.join([w for w in doc.split(" ") if w not in stopwords]) for doc in Tweet_rate['content_processed']]

# Lemmatization 
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer=WordNetLemmatizer()
Tweet_rate['content_processed'] = [' '.join([lemmatizer.lemmatize(w) for w in doc.split(" ")]) for doc in Tweet_rate['content_processed']]

In [4]:
df_data = Tweet_rate[["date", "Label", 'content_processed']]
len(df_data)
df_data.head()

,date,Label,content_processed
1,05/05/2009,neutral,appearing view tomorrow morning discus celebri...
2,05/08/2009,neutral,read top financial tip david letterman tinyurl...
3,05/08/2009,neutral,blog post celebrity apprentice finale lesson l...
4,05/12/2009,up,persona wallflower build wall cling
5,05/12/2009,neutral,miss usa tara conner fired believer chance


In [5]:
df_data = data_preprocess(df_data, 'content_processed', clean_data=True, remove_stopwords=False, sentiment=True, stem_lemma=False)

C:\Users\User\Documents\ISSS617-Python\Project-python\data_preprocess.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[CLEAN_COL] = clean_col(df[column])
C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\User\Documents\ISSS617-Python\Project-python\data_preprocess.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

# Training and Testing Split (based on date)

In [6]:
train_df = df_data[22500:29000]
test_df = df_data[29000: ]

In [7]:
train_df.head()

,date,Label,content_processed,clean_data,polarity,subjectivity
22503,12/24/2015,neutral,hillary deplore tone inflammatory rhetoric cam...,hillary deplore tone inflammatory rhetoric cam...,-0.6,1.00
22504,12/24/2015,neutral,ihatematt hillary allowed run criminal question,ihatematt hillary allowed run criminal question,-0.4,0.55
22505,12/24/2015,neutral,ihatematt hillary allowed run criminal,ihatematt hillary allowed run criminal,-0.4,0.55
22506,12/24/2015,neutral,christmas movie perfect tbt makeamericagreatag...,christmas movie perfect tbt makeamericagreatag...,1.0,1.00
22507,12/24/2015,neutral,classyexplorer gatewaypundit jim fail mention ...,classyexplorer gatewaypundit jim fail mention ...,-0.5,0.30


In [8]:
print(len(train_df))
print(len(test_df))

6500
3467


In [9]:
print('train data distribution:\n', train_df.Label.value_counts())
print('test data distribution:\n', test_df.Label.value_counts())

train data distribution:
 neutral    5419
up          547
down        534
Name: Label, dtype: int64
test data distribution:
 neutral    3021
down        272
up          174
Name: Label, dtype: int64


In [11]:
train_data, transform = data_vect_fit(train_df, CLEAN_COL, sentiment=True,topic_vect=True)
test_data = data_vect_transform(test_df, CLEAN_COL, transform )

(6500, 21)
(6500, 23)
(6500, 43)


C:\Users\User\Documents\ISSS617-Python\Project-python\model_python.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['polarity'] = df[column].apply(lambda x: TextBlob(x).sentiment.polarity)
C:\Users\User\Documents\ISSS617-Python\Project-python\model_python.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['subjectivity'] = df[column].apply(lambda x: TextBlob(x).sentiment.subjectivity)


In [12]:
train_label = train_df['Label']
test_label = test_df['Label']

In [13]:
print("training data shape: ", train_data.shape)
print("testing data shape: ", test_data.shape)

training data shape:  (6500, 43)
testing data shape:  (3467, 43)


# Model Training

In [17]:
clf = RandomForestClassifier()

In [29]:
print('random forest results: ')
clf = model_fit_predict(clf, train_data, train_label, test_data, test_label)

random forest results: 
training accuracy:  0.9732307692307692
testing accuracy:  0.8641476781078743


ValueError: multiclass format is not supported

In [30]:
clf.predict_proba(tf_idf.transform([test]))

NameError: name 'tf_idf' is not defined

# Save Model

In [ ]:
from model import save_transform
import pickle

In [ ]:
dump(clf, 'rf.m')
save_transform(transform, 'transform.m')